Homework 3.2 Naive Bayes
== 
***
<b>STEP 1: Loading the data</b>

Source of the data: Bureau of Transportation Statistics

https://www.transtats.bts.gov/DataIndex.asp
Airline On-Time Performance

In [1]:
setwd("C:/Users/Allie/Desktop/R")

In [2]:
flights <- read.csv("flights_2016_12.csv", stringsAsFactors = FALSE)

<b>STEP 2: Exploring and preparing the data</b>

The dataset contains the records of the domestic flights occured in the US over the period of December 2016. Originally constructed dataset contains 25 variables (columns) and 460,949 observations (rows). 

In [3]:
str(flights)

'data.frame':	460949 obs. of  25 variables:
 $ Year              : int  2016 2016 2016 2016 2016 2016 2016 2016 2016 2016 ...
 $ Quarter           : int  4 4 4 4 4 4 4 4 4 4 ...
 $ Month             : int  12 12 12 12 12 12 12 12 12 12 ...
 $ DayofMonth        : int  18 19 20 21 22 23 24 26 27 28 ...
 $ DayOfWeek         : int  7 1 2 3 4 5 6 1 2 3 ...
 $ FlightDate        : chr  "12/18/2016" "12/19/2016" "12/20/2016" "12/21/2016" ...
 $ UniqueCarrier     : chr  "AA" "AA" "AA" "AA" ...
 $ Carrier           : chr  "AA" "AA" "AA" "AA" ...
 $ Origin            : chr  "MSP" "MSP" "MSP" "MSP" ...
 $ Dest              : chr  "ORD" "ORD" "ORD" "ORD" ...
 $ CRSDepTime        : int  1558 1558 1558 1558 1558 1558 1605 1558 1558 1558 ...
 $ DepTime           : int  1556 1555 1555 1555 1600 1553 1628 1751 1605 1555 ...
 $ DepDelay          : int  -2 -3 -3 -3 2 -5 23 113 7 -3 ...
 $ DepDelayMinutes   : int  0 0 0 0 2 0 23 113 7 0 ...
 $ DepDel15          : int  0 0 0 0 0 0 1 1 0 0 ...
 $ CRSArrTime 

In [4]:
# Removing variables
drop <- c(1:3, 6:7, 12:15, 17:20, 23:24)
flights <- flights[,-drop]

In [5]:
# Removing all the cancelled flights from the dataset
flights <- flights[flights$Cancelled != 1, ]
# Removing the variable "Cancelled"
flightsFinal <- flights[, -9]

In [6]:
summary(flightsFinal)

   DayofMonth      DayOfWeek       Carrier             Origin         
 Min.   : 1.00   Min.   :1.000   Length:453485      Length:453485     
 1st Qu.: 8.00   1st Qu.:2.000   Class :character   Class :character  
 Median :16.00   Median :4.000   Mode  :character   Mode  :character  
 Mean   :16.07   Mean   :3.981                                        
 3rd Qu.:24.00   3rd Qu.:6.000                                        
 Max.   :31.00   Max.   :7.000                                        
                                                                      
     Dest             CRSDepTime     CRSArrTime   ArrivalDelayGroups
 Length:453485      Min.   :   5   Min.   :   1   Min.   :-2.0000   
 Class :character   1st Qu.: 915   1st Qu.:1109   1st Qu.:-1.0000   
 Mode  :character   Median :1325   Median :1520   Median :-1.0000   
                    Mean   :1330   Mean   :1494   Mean   : 0.0343   
                    3rd Qu.:1735   3rd Qu.:1920   3rd Qu.: 0.0000   
                  

In [7]:
# Binning DayofMonth variable
flightsFinal$DayofMonth = ordered(cut(flightsFinal$DayofMonth, c(0, 23, Inf), right = FALSE), 
                                  labels = c("Regular", "Holiday"))

In [8]:
summary(flightsFinal$DayofMonth)

Regular Holiday 
 322344  131141

In [9]:
str(flightsFinal$DayofMonth)

 Ord.factor w/ 2 levels "Regular"<"Holiday": 1 1 1 1 1 2 2 2 2 2 ...


In [10]:
flightsFinal$DayOfWeek = as.character(flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("1", "Monday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("2", "Tuesday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("3", "Wednesday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("4", "Thursday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("5", "Friday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("6", "Saturday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = gsub("7", "Sunday", flightsFinal$DayOfWeek)
flightsFinal$DayOfWeek = factor(flightsFinal$DayOfWeek)
str(flightsFinal$DayOfWeek)

 Factor w/ 7 levels "Friday","Monday",..: 4 2 6 7 5 1 3 2 6 7 ...


In [11]:
# Binning CRSDepTime (scheduled departure time)
flightsFinal$CRSDepTime = ordered(cut(flightsFinal$CRSDepTime, c(0, 600, 900, 1200, 1500, 1800, 2100, Inf), right = FALSE), 
                                  labels = c("Overnight", "Early Morning", "Morning", "Afternoon", "Late Afternoon", "Evening", "Late Evening"))
summary(flightsFinal$CRSDepTime)

Overnight  Early Morning        Morning      Afternoon Late Afternoon 
         12413          91222          84870          81238          83560 
       Evening   Late Evening 
         70460          29722

In [12]:
# Binning CRSArrTime (scheduled arrival time)
flightsFinal$CRSArrTime = ordered(cut(flightsFinal$CRSArrTime, c(0, 600, 900, 1200, 1500, 1800, 2100, Inf), right = FALSE), 
                                  labels = c("Night", "Early Morning", "Morning", "Afternoon", "Late Afternoon", "Evening", "Late Evening"))
summary(flightsFinal$CRSArrTime)

Night  Early Morning        Morning      Afternoon Late Afternoon 
         13009          42562          79936          81992          83495 
       Evening   Late Evening 
         82115          70376

In [13]:
# Binning ArrivalGroups variable
flightsFinal$ArrivalDelayGroups = ordered(cut(flightsFinal$ArrivalDelayGroups, c(-2, 1, 4, 12, Inf), right = FALSE), 
                                          labels = c("On-Time Arrival", "Moderately Late", "Very Late", "Severely Late"))
summary(flightsFinal$ArrivalDelayGroups)

On-Time Arrival Moderately Late       Very Late   Severely Late            NA's 
         348636           70135           27498            5960            1256

In [14]:
# Binning DistanceGroup variable
flightsFinal$DistanceGroup = ordered(cut(flightsFinal$DistanceGroup, c(1, 5, 11, Inf), right = FALSE), 
                                     labels = c("Short-Haul", "Medium-Haul", "Long-Haul"))
summary(flightsFinal$DistanceGroup)

Short-Haul Medium-Haul   Long-Haul 
     311282      131792       10411

In [15]:
# Converting chr variables into factor
flightsFinal$Carrier <- factor(flightsFinal$Carrier)
flightsFinal$Origin <- factor(flightsFinal$Origin)
flightsFinal$Dest <- factor(flightsFinal$Dest)

In [16]:
str(flightsFinal)

'data.frame':	453485 obs. of  9 variables:
 $ DayofMonth        : Ord.factor w/ 2 levels "Regular"<"Holiday": 1 1 1 1 1 2 2 2 2 2 ...
 $ DayOfWeek         : Factor w/ 7 levels "Friday","Monday",..: 4 2 6 7 5 1 3 2 6 7 ...
 $ Carrier           : Factor w/ 12 levels "AA","AS","B6",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Origin            : Factor w/ 297 levels "ABE","ABI","ABQ",..: 200 200 200 200 200 200 200 200 200 200 ...
 $ Dest              : Factor w/ 297 levels "ABE","ABI","ABQ",..: 211 211 211 211 211 211 211 211 211 211 ...
 $ CRSDepTime        : Ord.factor w/ 7 levels "Overnight"<"Early Morning"<..: 5 5 5 5 5 5 5 5 5 5 ...
 $ CRSArrTime        : Ord.factor w/ 7 levels "Night"<"Early Morning"<..: 5 5 5 5 5 5 5 5 5 5 ...
 $ ArrivalDelayGroups: Ord.factor w/ 4 levels "On-Time Arrival"<..: 2 2 1 1 1 2 2 3 1 1 ...
 $ DistanceGroup     : Ord.factor w/ 3 levels "Short-Haul"<"Medium-Haul"<..: 1 1 1 1 1 1 1 1 1 1 ...


In [17]:
# Splitting the data into Training and Test sets, containing 75% and 25% of observaions respectively
library(caret)
TrainingIndex <- createDataPartition(flightsFinal$ArrivalDelayGroups, p=0.75, list = FALSE)
f_train <- flightsFinal[TrainingIndex,]
f_test <- flightsFinal[-TrainingIndex,]

Loading required package: lattice
Loading required package: ggplot2


<b>STEP 3: Training a model on the data</b>

We need to install and load *e1071* package to build the classifier. The predicted class is ArrivalDelayGroups; the labels are: On-Time Arrival, Moderately Late, Very Late, Severely Late.

In [18]:
library(e1071)
f_classifier <- naiveBayes(ArrivalDelayGroups ~ ., data = f_train)

<b>STEP 4: Evaluating model performance</b>

Making predictions using the classifier and calculating the Accuracy rate:

In [19]:
f_test_pred <- predict(f_classifier, f_test)

In [20]:
cm <-confusionMatrix(f_test_pred, f_test$ArrivalDelayGroups)
cm$overall

Accuracy          Kappa  AccuracyLower  AccuracyUpper   AccuracyNull 
    0.76780534     0.01715141     0.76533283     0.77026410     0.77093653 
AccuracyPValue  McnemarPValue 
    0.99388667     0.00000000

<b>STEP 5: Improving model performance using Laplace smoothing</b>

In [21]:
f_classifier2 <- naiveBayes(ArrivalDelayGroups ~ ., data = flightsFinal, laplace = 2)
f_test_pred2 <- predict(f_classifier2, flightsFinal[,-8])

In [22]:
# New confusion matrix
cm2 <-confusionMatrix(f_test_pred2, flightsFinal$ArrivalDelayGroups)
cm2$overall

Accuracy          Kappa  AccuracyLower  AccuracyUpper   AccuracyNull 
    0.76907938     0.01811581     0.76784831     0.77030701     0.77092800 
AccuracyPValue  McnemarPValue 
    0.99844950     0.00000000

The model Accuracy stayed roughly at 77%.

In [23]:
library(gmodels)
CrossTable(f_test_pred2, flightsFinal$ArrivalDelayGroups,
           prop.chisq = FALSE, prop.t = FALSE, prop.r = FALSE,
           dnn = c('predicted', 'actual'))


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Col Total |
|-------------------------|

 
Total Observations in Table:  452229 

 
                | actual 
      predicted | On-Time Arrival | Moderately Late |       Very Late |   Severely Late |       Row Total | 
----------------|-----------------|-----------------|-----------------|-----------------|-----------------|
On-Time Arrival |          346510 |           68816 |           26799 |            5826 |          447951 | 
                |           0.994 |           0.981 |           0.975 |           0.978 |                 | 
----------------|-----------------|-----------------|-----------------|-----------------|-----------------|
Moderately Late |            1426 |            1023 |             430 |              65 |            2944 | 
                |           0.004 |           0.015 |           0.016 |           0.011 |                 | 
----------------|-----------------|--